In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

c:\Users\zhufe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
class RecurrentNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.001):
        super(RecurrentNeuralNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.Wx = nn.Parameter(torch.randn(hidden_size, input_size) * torch.sqrt(torch.tensor(2.0 / (input_size + hidden_size))))
        self.Wh = nn.Parameter(torch.randn(hidden_size, hidden_size) * torch.sqrt(torch.tensor(1.0 / hidden_size)))
        self.Wy = nn.Parameter(torch.randn(output_size, hidden_size) * torch.sqrt(torch.tensor(2.0 / (hidden_size + output_size))))
        self.bh = nn.Parameter(torch.zeros(hidden_size, 1))
        self.by = nn.Parameter(torch.zeros(output_size, 1))

        self.learning_rate = learning_rate
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        self.mse_loss = nn.MSELoss()
        
    def forward(self, x):
        h_prev = torch.zeros(self.hidden_size, 1)
        self.y_pred, self.h_hist = [], [h_prev]

        for t in range(len(x)):
            h_t = self.tanh(self.Wx @ x[t].view(-1, 1) + self.Wh @ h_prev + self.bh)
            y_t = self.sigmoid(self.Wy @ h_t + self.by)
            self.y_pred.append(y_t)
            self.h_hist.append(h_t)
            h_prev = h_t
        return self.y_pred

    def print_mse(self, X, y):
        y_pred = torch.cat(self.forward(X)).view(-1)
        mse = self.mse_loss(y.view(-1), y_pred)
        print(f"MSE: {mse.item()}")

    def backward(self, x, y):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        optimizer.zero_grad()

        y_pred = self.forward(x)
        y_pred_tensor = torch.cat(y_pred).view(-1)
        loss = self.mse_loss(y.view(-1), y_pred_tensor)
        loss.backward()
        optimizer.step()


In [3]:
# Parameter split_percent defines the ratio of training examples
def get_train_test(url, split_percent=0.8):
    df = pd.read_csv(url, usecols=[1], engine='python')
    data = np.array(df.values.astype('float32'))
    scaler = MinMaxScaler(feature_range=(0, 1))
    data = scaler.fit_transform(data).flatten()
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data
 
sunspots_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv'
train_data, test_data, data = get_train_test(sunspots_url)

In [4]:
input_size, hidden_size, output_size = 1, 5, 1
self = RecurrentNeuralNetwork(input_size, hidden_size, output_size)

In [5]:
x = torch.tensor(train_data[:-1])  # Input data
y = torch.tensor(train_data[1:])  # Target data

In [7]:
for _ in range(10):
    self.forward(x)
    self.backward(x, y)
    self.print_mse(x,y)

MSE: 0.11539042741060257
MSE: 0.1110578253865242
MSE: 0.10689526796340942
MSE: 0.10303846746683121
MSE: 0.09956854581832886
MSE: 0.09650582820177078
MSE: 0.09382680803537369
MSE: 0.09148649871349335
MSE: 0.08943492919206619
MSE: 0.08762546628713608


In [44]:
import torch
import torch.nn as nn

class RecurrentNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.1):
        super(RecurrentNeuralNetwork, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

        self.learning_rate = learning_rate
        self.mse_loss = nn.MSELoss()

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        y_pred = self.sigmoid(self.fc(out))
        return out

    def print_mse(self, X, y):
        y_pred = self.forward(X)
        mse = self.mse_loss(y, y_pred)
        print(f"MSE: {mse.item()}")

    def backward(self, x, y):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        optimizer.zero_grad()

        y_pred = self.forward(x)
        loss = self.mse_loss(y, y_pred)
        loss.backward()
        optimizer.step()


In [45]:
input_size, hidden_size, output_size = 1, 5, 1
self = RecurrentNeuralNetwork(input_size, hidden_size, output_size)

In [46]:
torch_x = x.view(1, 2255, 1)
torch_y = y.view(1, 2255, 1)

In [48]:
for _ in range(10):
    self.forward(torch_x)
    self.backward(torch_x, torch_y)
    self.print_mse(torch_x, torch_y)

MSE: 0.0653841644525528
MSE: 0.058179743587970734
MSE: 0.05234533175826073
MSE: 0.04756656289100647
MSE: 0.0436125248670578
MSE: 0.040310878306627274
MSE: 0.037531137466430664
MSE: 0.03517318144440651
MSE: 0.03315925970673561
MSE: 0.03142833709716797
